In [ ]:
# For Google colab
# !git clone https://github.com/WryBin/Steam_pre_commit
# !mv Steam_pre_commit/* .

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import utils
import models



# 读取数据
# combi = pd.read_csv('/content/drive/MyDrive/Steam/combi.csv')
combi = pd.read_csv('Dataset/combi.csv')



# 数据集划分
Train_data = combi[(combi['Time'] > '2021-12-20 08:00:00') & (combi['Time'] < '2021-12-22 23:30:00')]
pred_data = combi[combi['Time'] >= '2021-12-22 23:30:00']

# train_data, val_data = train_test_split(Train_data, test_size=0.2, random_state=42)


# # 特征选择
# features = ['2wind', 'SO2','NOx', 'Fan_speed', 'Grate_run','simu']
features = ['2wind', 'SO2','NOx', 'Fan_speed','simu']

if True:
  # redivision
  sample_weight, adv_cv_results = utils.adv_val(Train_data, pred_data, features)
  Train_data['sample_weight'] = sample_weight[:-1800]
  # train_data = Train_data[Train_data['sample_weight'] < 0.005]
  val_data = Train_data[Train_data['sample_weight'] >= 0.015]
  Train_data.loc[Train_data['sample_weight'] >= 0.015, 'sample_weight'] = None
  Train_data['Is_Train'] = Train_data.loc[:, 'sample_weight'].rolling(200, center=True).sum()
  train_data = Train_data[Train_data['Is_Train'] > 0]

  print('train_data:', train_data.shape)
  print('val_data:', val_data.shape)

In [ ]:
# 参数设置
Mode = 'train'

if Mode == 'pred':
    train_data = pd.concat([train_data, val_data], axis=0)

# 模型选择及运行
# # linear_regression
# Y_train, Y_val, Y_pred, model = models.linear_regression(train_data, val_data, pred_data, features)

# # # linear_regression_cv
# model, scores = models.linear_regression_cv(train_data, features)

# LightGBM
Y_train, Y_val, Y_pred, model= models.LightGBM(train_data, val_data, pred_data, features, Mode)
lgb.plot_importance(model, max_num_features=21)

# # LightGBM_param_cv
# model= models.LightGBM_param_cv(train_data, val_data, pred_data, features, Mode)

# # XGBoost
# Y_train, Y_val, Y_pred, model= models.XGBoost(train_data, val_data, pred_data, features, Mode)

# # XGBoost_param_cv
# model= models.XGBoost_param_cv(train_data, val_data, pred_data, features, Mode)
# model.best_params_

# # Arima
# result = models.Arima(train_data, val_data, pred_data, features, Mode)

# # Elastic_net
# Y_train, Y_val, Y_pred, model = models.ElasticNet(train_data, val_data, pred_data, features, Mode)

# # Elastic_net_cv
# model = models.ElascicNet_cv(train_data, val_data, pred_data, features, Mode)

In [ ]:
print('train_error:', mean_squared_error(train_data['Steam_flow'], Y_train, squared=False))
print('val_error', mean_squared_error(val_data['Steam_flow'], Y_val, squared=False))

plt.figure(figsize=(20, 10))
plt.plot(np.arange(len(train_data)), train_data['Steam_flow'], color='black')
plt.plot(np.arange(len(train_data)), Y_train, color='blue')
plt.ylabel('Steam_flow')
plt.legend(['真实值','预测值'],loc='upper right')

plt.figure(figsize=(20, 10))
plt.plot(np.arange(len(val_data)), val_data['Steam_flow'], color='black')
plt.plot(np.arange(len(val_data)), Y_val, color='blue')
plt.ylabel('Steam_flow')
plt.legend(['真实值','预测值'],loc='upper right')
plt.grid()
plt.show()

plt.figure(figsize=(20, 10))
plt.plot(np.arange(len(pred_data)), Y_pred)


In [ ]:
pred_data['ID'] = np.arange(1, len(pred_data)+1)
pred_data['Steam_flow'] = Y_pred
pred_data = pred_data[['ID', 'Time', 'Steam_flow']]
pred_data.to_csv("result.csv", index=False, sep=',')